In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from dt import DecisionTree

In [ ]:
from sklearn.model_selection import RepeatedKFold, train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score 
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.datasets import load_digits
data = load_digits()
X, y = data.data, data.target

print(f'{X.shape=}')

X[0,:].reshape([8,8])
print(y)

In [ ]:
f, axes = plt.subplots(1, 4, sharey=True, figsize=(16,6))
for i in range(4):
    axes[i].imshow(X[i,:].reshape([8,8]))

Делим выборку на обучающую и тестовую

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

Обучаем самодельное решающее дерево

In [ ]:
%%time
dt = DecisionTree(max_tree_depth=12, min_node_records=3)
dt.fit(x_train, y_train)

Оцениваем результат обучения

In [ ]:
y_train_pred = dt.predict(x_train)
y_test_pred = dt.predict(x_test)
accuracy_score(y_train_pred, y_train), accuracy_score(y_test_pred, y_test)

Обучаем решающее дерево из sklearn

In [ ]:
%%time
tree = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=3, max_depth=12, random_state=177)
tree.fit(x_train, y_train)

Оцениваем результат обучения

In [ ]:
tree_train_pred = tree.predict(x_train)
tree_test_pred = tree.predict(x_test)
accuracy_score(tree_train_pred, y_train), accuracy_score(tree_test_pred , y_test)

- Видно, что результаты сопоставимы, и как будто, у самодельного при прочих равных совсем чуть-чуть лучше точность на тестовой выборке, но это, скорее всего случайность, и при правильно подобраных параметрах (с помощью поиска по сетке) для обоих классификаторов реализация из sklearn должна, конечно, превзойти. 
- Кроме того, обучение самодельного решающего дерева происходит на несколько порядков дольше.

Дальше реализовал кросс-валидацию и поиск лучших гиперпараметров по сетке с помощью кросс валидации, но отладить пока не успел :(

### Подбор гиперпараметров

Реализация поиска по сетке с оценкой качетсва при помощи кросс-валидации

In [ ]:
def cross_validation(estimator_factory, X, y, cv=5, n_repeats=3):
    kf = RepeatedKFold(n_splits=cv, n_repeats=n_repeats, random_state=None) 

    metrics = []
    for train_index, test_index in kf.split(X):
        X_trn, X_tst = X[train_index], X[test_index] 
        y_trn, y_tst = y[train_index], y[test_index]

        estimator = estimator_factory()
        estimator.fit(X_trn, y_trn)

        y_prd = estimator.predict(X_tst)

        accuracy = accuracy_score(y_prd, y_tst)
        metrics.append(accuracy)

    return np.array(metrics)

def grid_search(X, y, max_tree_depths, min_node_records):
    best_accuracy = 0.0
    best_max_tree_depth = None
    best_min_node_records = None

    for mtd in max_tree_depths:
        for mnr in min_node_records:
            print(f'trying {mtd=}, {mnr=}')
            accuracy = np.mean(cross_validation(lambda: DecisionTree(max_tree_depth=mtd, min_node_records=mnr), X, y, cv=5, n_repeats=1))
            print(f'accuracy: {accuracy}')
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_max_tree_depth = mtd
                best_min_node_records = mnr

    return {'best_params' : (best_max_tree_depth, best_min_node_records), 'best_score' : best_accuracy }

Проверим кросс-валидацию на уже обученных моделях с параметрами, взятыми от балды

In [ ]:
np.mean(cross_validation(lambda: DecisionTree(max_tree_depth=12, min_node_records=3), x_train, y_train, cv=5, n_repeats=1))

In [ ]:
np.mean(cross_val_score(tree, x_train, y_train, cv=5, scoring='accuracy'))

Подберем гиперпараметры при помощи поиска по сетке

In [ ]:
max_tree_depths = [1, 3, 5, 7, 9, 11]
min_node_records = [1, 3, 5, 7, 9, 11]

grid_search(x_train, y_train, max_tree_depths, min_node_records)

In [ ]:
tree_params = {'min_samples_leaf': [1, 3, 5, 7, 9], 'max_depth': [1, 3, 5, 7, 9]}
tree_grid = GridSearchCV(tree, tree_params, cv=5, n_jobs=-1, verbose=True)

tree_grid.fit(x_train, y_train)
tree_grid.best_params_, tree_grid.best_score_

Видно, что результат подбора гиперпараметров для самодельного решающего дерева и реализации из sklearn совпадают, хотя, как и ожидалось, точность самодельного на оптимальных гиперпараметрах несколько меньше. Это лишний раз говорит о том, что самодельный алгоритм работает примерно так же, как реализация в sklearn, значит грубых ошибок не допущено.

### Обучение и классификация (при лучших гиперпараметрах)

Теперь обучим модели при оптимальных параметрах и проверим работу на тестовой выборке

In [ ]:
%%time
dt = DecisionTree(max_tree_depth=9, min_node_records=1)
dt.fit(x_train, y_train)

y_train_pred = dt.predict(x_train)
y_test_pred = dt.predict(x_test)
accuracy_score(y_train_pred, y_train), accuracy_score(y_test_pred, y_test)

In [ ]:
%%time
tree = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=1, max_depth=9)
tree.fit(x_train, y_train)

tree_train_pred = tree.predict(x_train)
tree_test_pred = tree.predict(x_test)
accuracy_score(tree_train_pred, y_train), accuracy_score(tree_test_pred , y_test)

Видно, что для обоих моделей достигается приблизительно та же точность, что и во время подбора гиперпараметров

### Вывод
Решающее дерево -- довольно простой алгоритм, который, однако, может довольно неплохо решать такие, довольно сложные, задачи, как распознавание рукописных символов, показывая при этом приемлемое качество классификации.